<a href="https://colab.research.google.com/github/thingumajig/qa-prototype/blob/master/qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Init

## Install packages

In [0]:
!pip install spacy

!pip3 uninstall --quiet --yes tensorflow
!pip3 install --quiet tensorflow-gpu==1.14.0
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet sentencepiece==0.1.83
!pip3 install --quiet tf-sentencepiece==0.1.83
!pip3 install --quiet simpleneighbors


     |████████████████████████████████| 377.0MB 46kB/s 
     |████████████████████████████████| 3.2MB 32.5MB/s 
     |████████████████████████████████| 491kB 39.7MB/s 
     |████████████████████████████████| 1.0MB 2.8MB/s 
     |████████████████████████████████| 2.7MB 2.6MB/s 
     |████████████████████████████████| 645kB 3.9MB/s 


## Initialize grammar parser

In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")
# doc = nlp("With respect to all losses caused by the peril of Flood, the Company shall not be liable, in the aggregate for any one Policy year, for more than its proportionate share of US$25,000,000.")
# doc = nlp("The Program limit of liability is US$400,000,000.")
# print(doc)
# for token in doc:
#     print("{2}-{1}({3}-{6}, {0}-{5})".format(token.text, token.tag_, token.dep_, token.head.text, token.head.tag_, token.i+1, token.head.i+1))
# for np in doc.noun_chunks:
#   print(np.text)


from spacy.symbols import *

np_labels = set([nsubj, nsubjpass, dobj, iobj, pobj]) # Probably others too
np_labels_full = set([nsubj, nsubjpass, dobj, iobj, pobj, csubj, csubjpass, attr]) # Probably others too
# print(dir(spacy.symbols))
# subj - subject
# nsubj - nominal subject
# nsubjpass - passive nominal subject
# csubj - clausal subject
# csubjpass - passive clausal subject

def iter_nps(doc):
    for word in doc:
        if word.dep in np_labels_full:
            yield word

def iter_nps_str(doc):
  s = ''
  for np in iter_nps(doc):
    for t in np.subtree:
      s += str(t)+' '
    yield s.strip()
    s = ''


# print('='*20)
# for np in iter_nps(doc):
#    print(np)
#    for t in np.subtree:
#      print(f'\t{t}')

# print('='*20)
# for np in iter_nps_str(doc):
#   print(np)


# print('='*20)
# for token in doc:
#   print(f'{token.text} {token.tag_} {token.dep_} {str(token.dep)} \t\t\thead: {token.head.tag_} {token.head.dep_} {token.head.text}')     
#   for t in token.subtree:
#     print(f'\t{t}')


##  Set up Tensorflow graph

In [0]:
%%time
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

# Set up graph.
g = tf.Graph()
with g.as_default():
  questions_input = tf.placeholder(dtype=tf.string, shape=[None])
  responses_input = tf.placeholder(dtype=tf.string, shape=[None])
  contexts_input = tf.placeholder(dtype=tf.string, shape=[None])


  module = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/1")
  question_embeddings = module(
    dict(input=questions_input),
    signature="question_encoder", as_dict=True)

  response_embeddings = module(
    dict(input=responses_input,
         context=contexts_input),
    signature="response_encoder", as_dict=True)

  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


CPU times: user 35.9 s, sys: 1.89 s, total: 37.8 s
Wall time: 38.8 s


# Run

##Initialize tensorflow session.

In [0]:
%%time
# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)

CPU times: user 10.5 s, sys: 2 s, total: 12.5 s
Wall time: 14.3 s


## Tests

In [0]:
# %%time
sentences = '''
F.	PROGRAM LIMITS OF LIABILITY

(1)	The Program limit of liability is US$400,000,000. 

(2)	Sublimits below are applicable to all direct physical loss, damage or destruction insured against, except an Accident. 

(a)	With respect to all losses caused by the peril of Flood, the Company shall not be liable, in the aggregate for any one Policy year, for more than its proportionate share of US$25,000,000. 

But not to exceed US$5,000,000 in the aggregate for any one Policy year
for the peril of Flood occurring in Special Flood Hazard Areas. However,
this Special Flood Hazard Area sublimit shall not apply to loss involving 
personal property, buildings, or structures wholly located outside an area
designated as a Special Flood Hazard Area.

The aggregate limit stated in the paragraph above shall be part of the overall aggregate limit stated in the introduction to this paragraph.

Even if the peril of Flood is the predominant cause of direct physical loss, damage or destruction, any ensuing physical loss, damage or destruction arising from a peril not otherwise excluded herein shall not be subject to the sublimit or aggregate specified in this paragraph (a).
'''
sentence1 = 'With respect to all losses caused by the peril of Earthquake, the Company shall not be liable, in the aggregate for any one Policy year, for more than its proportionate share of US$50,000,000.'

sentence2 = '''
Суд освободил из-под стражи следователя Алексея Шиманского и полицейского Кирилла Лисового из Санкт-Петербурга, которых подозревают в посредничестве при передаче взятки в 19 миллионов рублей. Об этом сообщает «Фонтанка».
'''

import simpleneighbors

def  qa(sentences, queries, use_parser = True):
  index = simpleneighbors.SimpleNeighbors(
      512, metric='angular')

  def candidate_generation(t, from_n=1, to_n=6):
    l = t.split()
    grams = []
    for n in range(from_n, to_n):
      grams += [' '.join(l[i:i+n]) for i in range(len(l)-n+1)]
    
    return grams


  def display_nearest_neighbors(query_text):
    print(f'Query: {query_text}')
    query_embedding = session.run(question_embeddings, feed_dict={questions_input: [query_text]})['outputs'][0]
    # print("query_embedding:")
    # print(query_embedding)
    search_results = index.nearest(query_embedding, n=10)
    print('Top-10 Responses:')
    for s in search_results:
      print(f'\t{s}')


  def calculate_sentences_emb(sentences, use_parser=True):
    responses = []
    contexts = []
    if use_parser:
      doc = nlp(sentences)
      for sent in doc.sents:
        # print('='*40)
        # print(sent)
        # print('-- Noun groups & attributes: --')
        for np in iter_nps_str(sent):
          # print(f'\t{np}')
          responses.append(str(np).strip())
          contexts.append(str(sent).strip())
    else:
      grams = candidate_generation(sentences, 1, 5)
      for g in grams:
        responses.append(g.strip())
        contexts.append(sentences.strip())


    candidate_embeddings = session.run(
        response_embeddings,
        feed_dict={
            responses_input: responses,
            contexts_input: contexts
        })
    print(f'Shape: {candidate_embeddings["outputs"].shape}')
    # print(candidate_embeddings["outputs"][0])
    print('Candidates:')
    for i in range(len(responses)):
      print(f'\t{responses[i]}')
      index.add_one(responses[i], candidate_embeddings['outputs'][i])

    index.build()
    
  calculate_sentences_emb(sentences, use_parser)
  # display_nearest_neighbors('Какие имена у освобождённых')
  if isinstance(queries, list):
    for q in queries:
      display_nearest_neighbors(q)
  else:
      display_nearest_neighbors(queries)



# qa(sentences, 'имена отпущенных')
qa(sentence1, ['peril','sum'])
qa(sentence2, ['имена отпущенных', 'сумма взятки'])
  

Shape: (10, 512)
Candidates:
	respect to all losses caused by the peril of Earthquake
	all losses caused by the peril of Earthquake
	the peril of Earthquake
	Earthquake
	the Company
	the aggregate for any one Policy year
	any one Policy year
	more than its proportionate share of US$ 50,000,000
	its proportionate share of US$ 50,000,000
	US$ 50,000,000
Query: peril
Top-10 Responses:
	the peril of Earthquake
	respect to all losses caused by the peril of Earthquake
	all losses caused by the peril of Earthquake
	the Company
	Earthquake
	any one Policy year
	US$ 50,000,000
	more than its proportionate share of US$ 50,000,000
	the aggregate for any one Policy year
	its proportionate share of US$ 50,000,000
Query: sum
Top-10 Responses:
	more than its proportionate share of US$ 50,000,000
	its proportionate share of US$ 50,000,000
	the aggregate for any one Policy year
	US$ 50,000,000
	Earthquake
	the Company
	respect to all losses caused by the peril of Earthquake
	the peril of Earthquake
	al

# Form

In [0]:
%%time

#@title Question and answer
text = "\u0410\u0441\u0442\u0440\u043E\u043D\u043E\u043C\u044B \u0418\u043D\u0441\u0442\u0438\u0442\u0443\u0442\u0430 \u0432\u043D\u0435\u0437\u0435\u043C\u043D\u043E\u0439 \u0444\u0438\u0437\u0438\u043A\u0438 \u041E\u0431\u0449\u0435\u0441\u0442\u0432\u0430 \u041C\u0430\u043A\u0441\u0430 \u041F\u043B\u0430\u043D\u043A\u0430 \u0432 \u0413\u0435\u0440\u043C\u0430\u043D\u0438\u0438 \u0441\u043E\u043E\u0431\u0449\u0438\u043B\u0438 \u043E \u0432\u0441\u043F\u043B\u0435\u0441\u043A\u0435 \u0430\u043A\u0442\u0438\u0432\u043D\u043E\u0441\u0442\u0438 \u043D\u0435\u0438\u0437\u0432\u0435\u0441\u0442\u043D\u043E\u0433\u043E \u0438\u0441\u0442\u043E\u0447\u043D\u0438\u043A\u0430 \u0440\u0435\u043D\u0442\u0433\u0435\u043D\u043E\u0432\u0441\u043A\u0438\u0445 \u043B\u0443\u0447\u0435\u0439 \u0432 \u0433\u0430\u043B\u0430\u043A\u0442\u0438\u043A\u0435 NGC 300, \u0440\u0430\u0441\u043F\u043E\u043B\u043E\u0436\u0435\u043D\u043D\u043E\u0439 \u0432 \u0441\u0435\u043C\u0438 \u043C\u0438\u043B\u043B\u0438\u043E\u043D\u0430\u0445 \u0441\u0432\u0435\u0442\u043E\u0432\u044B\u0445 \u043B\u0435\u0442 \u043E\u0442 \u0417\u0435\u043C\u043B\u0438." #@param {type:"string"}
query = "\u043D\u0430 \u043A\u0430\u043A\u043E\u043C \u0440\u0430\u0441\u0441\u0442\u043E\u044F\u043D\u0438\u0438?" #@param {type:"string"}
use_token_ngram = True #@param {type:"boolean"}

qa(text, [query], not use_token_ngram)


Shape: (110, 512)
Candidates:
	Астрономы
	Института
	внеземной
	физики
	Общества
	Макса
	Планка
	в
	Германии
	сообщили
	о
	всплеске
	активности
	неизвестного
	источника
	рентгеновских
	лучей
	в
	галактике
	NGC
	300,
	расположенной
	в
	семи
	миллионах
	световых
	лет
	от
	Земли.
	Астрономы Института
	Института внеземной
	внеземной физики
	физики Общества
	Общества Макса
	Макса Планка
	Планка в
	в Германии
	Германии сообщили
	сообщили о
	о всплеске
	всплеске активности
	активности неизвестного
	неизвестного источника
	источника рентгеновских
	рентгеновских лучей
	лучей в
	в галактике
	галактике NGC
	NGC 300,
	300, расположенной
	расположенной в
	в семи
	семи миллионах
	миллионах световых
	световых лет
	лет от
	от Земли.
	Астрономы Института внеземной
	Института внеземной физики
	внеземной физики Общества
	физики Общества Макса
	Общества Макса Планка
	Макса Планка в
	Планка в Германии
	в Германии сообщили
	Германии сообщили о
	сообщили о всплеске
	о всплеске активности
	всплеске активности